In [1]:
# Credit: Adrian Rosebrock
# https://www.pyimagesearch.com/2015/03/30/accessing-the-raspberry-pi-camera-with-opencv-and-python/
 
# import the necessary packages


from picamera.array import PiRGBArray # Generates a 3D RGB array
from picamera import PiCamera # Provides a Python interface for the RPi Camera Module
import time # Provides time-related functions
import cv2 # OpenCV library
import numpy as np

# Initialize the camera
camera = PiCamera()
 
# Set the camera resolution
camera.resolution = (640, 480)
camera.start_preview() 
# Set the number of frames per second
camera.framerate = 16
 
# Generates a 3D RGB array and stores it in rawCapture
raw_capture = PiRGBArray(camera, size=(640, 480))
 
# Wait a certain number of seconds to allow the camera time to warmup
time.sleep(0.1)
ref = 317.1*50
prev=0
xinit=0
yinit=0
rinit=0
desiredz = 400
# Capture frames continuously from the camera
counter=0
counterdet=0
counternone=0
vx= 0
vz= 0
for frame in camera.capture_continuous(raw_capture, format="bgr", use_video_port=True):
    counter+=1
    # Grab the raw NumPy array representing the image
    image = frame.array
    img_hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    # define range of blue color in HSV
    lower = np.array([75,5,75])
    upper = np.array([105,100,255])

    # Threshold the HSV image to get only blue colors
    mask = cv2.inRange(img_hsv, lower, upper)

    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(image,image, mask= mask)

    #image=res
    

    # Display the frame using OpenCV
    #cv2.imshow("Frame", image)
    image = cv2.medianBlur(image,7)
    #image = cv2.GaussianBlur(image,(5,5),cv2.BORDER_DEFAULT)
    
    output = image.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    (thresh, img_bw) = cv2.threshold(gray, 128, 255,cv2.THRESH_OTSU)
    #bw = cv2.threshold(gray, 125, 255, cv2.THRESH_BINARY)[1]
    #cv2.imshow('org',image)
    #cv2.imshow('gray',gray)
    #cv2.imshow('bw',img_bw)
    #cv2.waitKey(0) 
    #cv2.destroyAllWindows()

    rows, cols, ch = image.shape
    xcentre = int(cols/2)
    ycentre = int(rows/2)
    #print(rows,cols,ch,xcentre,ycentre)
    #gray = auto
    flag = True
    # detect circles in the image
    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, 200,param1=70,param2=30,minRadius=0,maxRadius=125)
    # ensure at least some circles were found
    #print(circles)
    if circles is not None:

        #print(circles.size)
        # convert the (x, y) coordinates and radius of the circles to integers
        circles = np.round(circles[0, :]).astype("int")
        # loop over the (x, y) coordinates and radius of the circles
        for (x, y, r) in circles:
            #print(x,y,r)
            # draw the circle in the output image, then draw a rectangle
            # corresponding to the center of the circle
    #         if ~flag:
    #             break
    #         if flag:
    #             flag = False
    #         print('1')
            if flag:
                cv2.circle(output, (x, y), r, (0, 255, 0), 4)
                cv2.rectangle(output, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)
                xinit=x
                yinit=y
                rinit=r
            flag = False


        # show the output image
        
        cv2.rectangle(output, (xcentre - 5, ycentre - 5), (xcentre + 5, ycentre + 5), (0, 0, 255), -1)
        #cv2.imshow('start',gray)
        cv2.imshow("output", np.hstack([image, output])
)
        
    # Wait for keyPress for 1 millisecond
    else:
        cv2.imshow("output", image)
        
    if(not(circles is None)):
        counterdet+=1
        print(ref/r)
        prev=ref/r
        vx = -(1.1*(xinit-xcentre) + vx*0.2)
        vz = (ref/r - desiredz)*0.6 + vz*0.2
    
        #print("Left")
    else:
         counternone+=1
    f=open("/home/pi/Desktop/Python code/instructions.txt",'w')
#     print(vx)
#     print(vz)
    f.write(str(vx)[0:6] + "x" + str(vz)[0:6])
    f.close()
   
    key = cv2.waitKey(20) & 0xFF
    
   
    
    
 
    # Clear the stream in preparation for the next frame
    raw_capture.truncate(0)
     
    # If the `q` key was pressed, break from the loop
    if key == ord("q"):
        
        camera.close()
        cv2.destroyAllWindows()
        vx=0.0
        vz=0.0
        f=open("/home/pi/Desktop/Python code/instructions.txt",'w')
        f.write(str(vx)[0:6] + "x" + str(vz)[0:6])
        f.close()
        print(counterdet)
        print(counternone)
        break
    #print(counter%20)    
    if counter%20==0:
        vx= 0.00
        
        vz= 0.00
        
#         camera.close()
#         camera = PiCamera()
#         camera.resolution = (640, 480)
#         camera.framerate = 16
#         raw_capture = PiRGBArray(camera, size=(640, 480))
        
        

ModuleNotFoundError: No module named 'picamera'

In [17]:
camera.close()